In [4]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Wczytanie danych
df = pd.read_csv(r"C:\Users\tomas\Desktop\Praca dyplomowa\Tweets_Kaggle_Clean.csv")

# Pobranie listy stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Lista zbędnych słów (linie lotnicze, słowa kluczowe)
extra_stopwords = {"flight", "flights", "http", "https", "airline", "plane", 
                   "americanair", "jetblue", "united", "southwestair", "usairways", "virginamerica"}

# Funkcja do czyszczenia tekstu
def clean_text(text):
    words = text.lower().split()
    words = [word for word in words if word not in stop_words and word not in extra_stopwords]
    return " ".join(words)

# Czyszczenie danych
df["clean_text"] = df["text"].apply(clean_text)

# Tworzymy wektor cech TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X = vectorizer.fit_transform(df["clean_text"])

# Konwersja sentymentu na liczby (0 = negatywny, 1 = neutralny, 2 = pozytywny)
y = df["airline_sentiment"].map({"negative": 0, "neutral": 1, "positive": 2})

# Podział na zestaw treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Trenowanie modelu regresji logistycznej
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Predykcja na zbiorze testowym
y_pred = model.predict(X_test)

# Ocena modelu
print("\n📊 Raport klasyfikacji:\n", classification_report(y_test, y_pred))
print("\n🎯 Dokładność modelu:", accuracy_score(y_test, y_pred))

# Analiza wpływowych słów
feature_names = vectorizer.get_feature_names_out()
class_labels = ["negative", "neutral", "positive"]

for i, label in enumerate(class_labels):
    top_words = np.argsort(model.coef_[i])[-10:]  # 10 najważniejszych słów
    print(f"\n🔹 Najbardziej wpływowe słowa dla klasy '{label}':")
    print([feature_names[j] for j in top_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tomas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



📊 Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.81      0.94      0.87      1889
           1       0.66      0.46      0.54       580
           2       0.80      0.61      0.69       459

    accuracy                           0.79      2928
   macro avg       0.76      0.67      0.70      2928
weighted avg       0.78      0.79      0.78      2928


🎯 Dokładność modelu: 0.7909836065573771

🔹 Najbardziej wpływowe słowa dla klasy 'negative':
['hrs', 'fix', 'delay', 'luggage', 'cancelled', 'hour', 'hold', 'delayed', 'worst', 'hours']

🔹 Najbardziej wpływowe słowa dla klasy 'neutral':
['march', 'need', 'question', 'number', 'policy', 'united', 'americanair', 'southwestair', 'hi', 'jetblue']

🔹 Najbardziej wpływowe słowa dla klasy 'positive':
['thx', 'excellent', 'appreciate', 'love', 'best', 'amazing', 'awesome', 'great', 'thanks', 'thank']
